In [ ]:
# This project is to predict which topic of petitions may gain interests from citizens.

In [ ]:
# data preprocessing -> tokenizing & creating variables -> word embedding -> testing  -> TextCNN

In [2]:
import pandas as pd 
import numpy as np 

result = pd.read_csv("./datasets/crawling.csv")

In [3]:
print(result.shape)


df = result
df.head(3)
df.info()

(10881, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10881 entries, 0 to 10880
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  10881 non-null  object
 1   content   10881 non-null  object
 2   count     10881 non-null  object
 3   end       10881 non-null  object
 4   start     10881 non-null  object
 5   title     10881 non-null  object
dtypes: object(6)
memory usage: 510.2+ KB


In [4]:
print(df.loc[0]['content'])
print('---')
print(df.loc[1]['title'])

본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 발견하였다. 탈의실 천장에 CCTV를 발견한 것이다. 본인은 이에 경악하였다.

탈의실에 CCTV를 설치하는 것은 개인정보보호법 25조 2항을 위반한 사안으로 심각한 법적 문제이다. 촬영이 되었다면 이는 성폭력범죄의 처벌 등에 관한 특례법 14조 1항 위반이기도 하다. 또한, 법적인 문제일 뿐만 아니라 개인의 자유를 침해하고 성적 수치심을 불러일으키는 매우 심각한 윤리적 문제이다.
이렇듯 불법카메라는 그 자체로 굉장히 심각한 문제이다. 그런데 공공기관에서 탈의실에 불법, 비윤리적으로 카메라를 설치하였고, 이를 운영하였을지도 모르는 상황이 발생하였다. 도대체 한국 정부는 인권을 무엇이라고 생각하는 것인가? 

발견 직후 인권위에 진정을 넣었지만 인권위에서는 현재까지 조사가 되고 있는지, 진행은 어떻게 되었는지 어떠한 답변도 하지 않고 있다. 병무청측에서도 어떠한 답변도 없었다.

더 이상 답변을 기다리는 것이 의미 없다고 생각한 본인은 모 커뮤니티에 이 사실을 알렸다. 이후 이슈화가 되자 병무청은 설치는 되었으나 운영은 하지 않았다는 대답을 내놓았지만 이에 대한 조사 경과 발표나 사과, 책임 있는 대응은 전혀 보여주고 있지 않다.

이는 개인의 자유와 권리의 본질적 내용을 침해하는 강제징집, 국제노동기구가 인정한 강제징용/강제노동을 당하는 20대 남성들이 국가에 의해 신체를 감시당한 사건이다. 한국 정부는 병역의무자에게는 인권이 없다고 생각하는가?

언제까지 국가는 개인들을 노예 취급할 것인가? 몇백원도 안되는 시급을 주며 강제로 폭력 속에서 부리는 것도 모자라 이제는 이런 어처구니 없는 일까지 일어나는 것에 경악하지 않을 수 없다. 

이에 본인은 위 사실을 강력히 규탄하며 아래와 같은 대응을 요구한다.

1. 서울지방병무청 징병검사소 탈의실에 CCTV가 설치된 경위와 운영 여부, 관리 내역 등에 대해 낱낱이 조사하여 그 진상을 밝혀라.
2. 병무청장과 서울지방병무청장은 이 사건

In [ ]:
# data preprocessing

import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]',' ', str(text))
    return text

#for word embedding vector
def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)


In [6]:
# tokenizing 
# titles will be separated and saved by morphemes, contents will be handled by nouns.

from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)



In [7]:
df['token_final'] = df.title_token + df.content_token

# for number >=1000 , ',' must be deleted.
df['count'] = df['count'].replace({',':''}, regex=True).apply(lambda x : int(x))

print(df.dtypes) 

# 'Yes' if participants >= 1000
df['label'] = df['count'].apply(lambda x : 'Yes' if x>=1000 else 'No')

category         object
content          object
count             int64
end              object
start            object
title            object
title_token      object
content_token    object
token_final      object
dtype: object


In [8]:
# dataframe processed successfully
df.head()

,category,content,count,end,start,title,title_token,content_token,token_final,label
0,인권/성평등,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,267,2020-02-01,2020-01-02,서울지방병무청 탈의실에 설치된 에 대한 진상규명을 요구한다 또한 인권위의 미온적...,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...","[본인, 경, 서울, 지방, 병무청, 제, 검사, 탈의실, 수, 것, 발견, 탈의실...","[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...",No
1,경제민주화,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,271,2020-02-01,2020-01-02,주식시장 활성화 및 소액 개미 투자자 보호,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호]","[우리, 나라, 코스피, 총, 미국, 애플, 설, 돌, 정도, 한국, 주식시장, 평...","[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총...",No
2,행정,억울한 일로 국민청원을 신청합니다 저는 구치소 교도관입니다 ...,198,2020-02-01,2020-01-02,교정기관의 민낮,"[교정, 기관, 의, 민낮]","[일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이주, 전, 구치소, 독, 거...","[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이...",No
3,안전/환경,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다 우리 일상에서 가...,170,2020-02-01,2020-01-02,미세먼지 저감 대책,"[미세먼지, 저, 감, 대책]","[미세먼지, 심각, 성은, 이제, 적극, 대안, 요구, 우리, 일상, 가장, 적극,...","[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,...",No
4,교통/건축/국토,저는 우선 아이셋의 부모입니다 식구가 많은편이고 아이들이 성장함에 따라 집이 ...,2127,2020-02-01,2020-01-02,악질세입자 방지를 위한 세입자보호법을 재정해주세요,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...","[저, 우선, 아이, 셋, 부모, 식구, 편이, 아이, 집, 나름, 꿈, 가지, 좀...","[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...",Yes


In [9]:
df_drop = df[[
    'token_final', 'label'
]]

In [10]:
#!pip install gensim
import sys
sys.executable
# set up new virtual environment, installed python 3.8, gensim 3.8.3 to solve the problem
# finish downloading other pandas,konlpy.. be sure to check interpreter and kernel as well
# below is how to set up new virtual env with a different python version.
# https://stackoverflow.com/questions/70422866/how-to-create-a-venv-with-a-different-python-version

'c:\\Classes\\ml\\citizen_env\\Scripts\\python.exe'

In [11]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'],
                           sg=1, # skip-gram
                           size = 100,
                           window = 2,
                           min_count = 1,
                           workers = 4,
                           )
print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)



Word2Vec(vocab=42642, size=100, alpha=0.025)
[('음주', 0.8795356154441833), ('무면허', 0.8355539441108704), ('뺑소니', 0.8089871406555176), ('윤창', 0.7701980471611023), ('전과자', 0.7637375593185425), ('살인자', 0.75953209400177), ('운전자', 0.7551395893096924), ('살인죄', 0.7533096075057983), ('촉법소년', 0.7402899861335754), ('승자', 0.7366692423820496)]


In [12]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('datasets/petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format("datasets/petitions_tokens_w2v")

# most_similar shows the same result as before
model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('음주', 0.8795356154441833), ('무면허', 0.8355539441108704), ('뺑소니', 0.8089871406555176), ('윤창', 0.7701980471611023), ('전과자', 0.7637375593185425), ('살인자', 0.75953209400177), ('운전자', 0.7551395893096924), ('살인죄', 0.7533096075057983), ('촉법소년', 0.7402899861335754), ('승자', 0.7366692423820496)]


In [14]:
# dataset to train and validation set
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

#dataset is small, so test set is to be skipped.
tr.to_csv('datasets/train.csv', index=False, encoding="utf-8-sig")
val.to_csv('datasets/validation.csv', index=False, encoding="utf-8-sig")

In [1]:
# defining Field class
import torchtext
from torchtext.data import Field # pip install torchtext == 0.6  

def tokenizer(text):
    text=re.sub('[\][\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential=False)

In [6]:
import re
# create dataset in TEXT, LABEL format using TabularDataset.splits()
from torchtext.data import TabularDataset

train, validation = TabularDataset.splits(
    path= 'datasets/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)
print('Train:', train[0].text, train[0].label)
print('Validation:', validation[0].text, validation[0].label)

Train: ['농기', '계', '부가세', '환급', '도와주세요', '차체', '굴절', '농용', '로더', '부가세', '환급', '대한', '민원', '내용', '톤', '미만', '차체', '굴절', '로더', '농업', '기계로', '포함', '개정', '이유', '및', '내용', '농림축산식품부', '제공', '개정', '일', '사실', '건설', '용도', '사용', '차체', '굴절', '향', '치가', '자체', '중량', '톤', '미만', '타이어', '로더', '건설', '기계', '범위', '제외', '내용', '건설', '기계', '관리', '법', '시행', '령', '이', '개정', '해당', '로더', '농업', '기계', '범위', '포함', '구입', '거나', '그', '이용', '부대', '시설', '설치', '자', '등', '국가', '지방자치단체', '로부터', '자금', '등', '것임', '농업', '기계', '임', '불구', '현재', '부가세', '환급', '실정', '현재', '농업', '기계', '부가세', '관련', '면세', '환급', '농업', '기계', '농기', '포함', '농기', '구입', '모든', '혜택', '적용', '농업', '기계화', '촉진법', '농기', '범위', '농용', '로더', '톤', '미만', '차체', '굴절', '로더', '을', '포함', '다라', '명기', '이유', '톤', '미만', '부가세', '환급', '톤', '미만', '굴절', '로더', '해당', '건가', '법', '개정', '통해', '농기', '구매', '시', '농가', '부담', '가중', '해소', '농용', '로더', '범위', '법계', '정이', '시간', '현재', '굴절', '생산', '제조', '수입', '판매', '업체', '추세', '축산', '농가', '대형화', '해당', '제품', '수요', '농가', '현실', '정책', '세부', '정책', '부처',

In [7]:
# define vocab & DataLoader
import torch
from torchtext.vocab import Vectors

from torchtext.data import BucketIterator

vectors = Vectors(name="datasets/petitions_tokens_w2v")

TEXT.build_vocab(train,
                 vectors=vectors,
                 min_freq=1,
                 max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets= (train,validation),
    batch_size=8,
    device=device,
    sort=False
)

print("embedding vector's dimension:{}".format(TEXT.vocab.vectors.shape))


embedding vector's dimension:torch.Size([39040, 100])
